In [6]:
import os
import win32com.client
import pythoncom
import datetime, time
import sys

from init_Stock import *

##################################################
def __Init_Conn() :

    class XASessionEventHandler:
        login_state = 0

        def OnLogin(self, code, msg):
            if code == "0000":
                print("로그인 성공")
                XASessionEventHandler.login_state = 1
            else:
                print("로그인 실패")

    instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)

    instXASession.ConnectServer(Server_Name, 20001)
    instXASession.Login(id, passwd, cert_passwd, 0, 0)
    
    while XASessionEventHandler.login_state == 0:
        pythoncom.PumpWaitingMessages()

    num_account = instXASession.GetAccountListCount()
    for i in range(num_account):
        account = instXASession.GetAccountList(i)
        print("계좌번호 : %s" % (account))
        
##################################################
def __Check_Account() :

    # 주식잔고 조회2
    class XAQueryEventHandlerT0424:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT0424.query_state = 1

    instXAQueryT0424 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0424)
    instXAQueryT0424.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0424.res"

    instXAQueryT0424.SetFieldData("t0424InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT0424.SetFieldData("t0424InBlock", "passwd", 0, Account_PWD)
    instXAQueryT0424.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT0424.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count1 = instXAQueryT0424.GetBlockCount("t0424OutBlock1")
    print(count1)

    for i in range(count1):
        expcode = instXAQueryT0424.GetFieldData("t0424OutBlock1", "expcode", i)
        hname = instXAQueryT0424.GetFieldData("t0424OutBlock1", "hname", i) # 종목명
        janqty = instXAQueryT0424.GetFieldData("t0424OutBlock1", "janqty", i) # 잔고수량
        mdposqt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mdposqt", i) # 매도 가능수량
        pamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "pamt", i) # 평균단가
        mamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mamt", i) # 매입금액
        appamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "appamt", i) # 평가금액
        dtsunik = instXAQueryT0424.GetFieldData("t0424OutBlock1", "dtsunik", i) # 평가손익
        sunikrt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "sunikrt", i) # 수익율
        print("종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 매입액 : %s, 손액 : %s, 손익 : %s" % (expcode, hname, janqty, pamt, mamt, dtsunik, sunikrt))

##################################################
def __Check_Expcode(expcode) :

    # 주식잔고 조회2
    class XAQueryEventHandlerT3320:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT3320.query_state = 1

    instXAQueryT3320 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT3320)
    instXAQueryT3320.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t3320.res"

    instXAQueryT3320.SetFieldData("t3320InBlock", "gicode", 0, expcode) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT3320.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT3320.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count1 = instXAQueryT3320.GetBlockCount("t3320OutBlock1")
    print(count1)

    for i in range(count1):
        gicode = instXAQueryT3320.GetFieldData("t3320OutBlock1", "gicode", i) # 기업코드
        per = instXAQueryT3320.GetFieldData("t3320OutBlock1", "per", i) # PER
        pbr = instXAQueryT3320.GetFieldData("t3320OutBlock1", "pbr", i) # PBR
        roa = instXAQueryT3320.GetFieldData("t3320OutBlock1", "roa", i) # ROA
        roe = instXAQueryT3320.GetFieldData("t3320OutBlock1", "roe", i) # ROE
        t_gsym = instXAQueryT3320.GetFieldData("t3320OutBlock1", "t_gsym", i) # 최근 분기년도
        print("종목번호 : %s, PER : %s, PBR : %s, ROA : %s, ROE : %s, 최근분기 : %s" % (gicode, per, pbr, roa, roa, t_gsym))


##################################################
   
if __name__ == '__main__' :
    print("# Init Connection")
    __Init_Conn()
    print("# Check Expcode_1")
    __Check_Expcode("005930")
    print("# Check Expcode_2")
    print("# END")


# Init Connection
로그인 성공
계좌번호 : 00101753258
계좌번호 : 00146753258
계좌번호 : 10011702835
계좌번호 : 00445100633
계좌번호 : 20053468601
계좌번호 : 20053468605
계좌번호 : 20053468606
# Check Expcode_1
1
종목번호 : A005930, PER : 30.00, PBR : 2.28, ROA : 7.06, ROE : 7.06, 최근분기 : 202009
# Check Expcode_2
# END
